<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/Solar-Fullstack-LLM-101/01_hello_solar.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# 14. Reasoning

## Goal
...

In [80]:
! pip3 install -qU langchain-upstage requests python-dotenv datasets

## UPSTAGE_API_KEY
To obtain your Upstage API key, follow these steps:

1. Visit the Upstage AI console at <https://console.upstage.ai>.
2. Sign up for an account if you don't already have one.
3. Log in to your account.
4. Navigate to the API key section.
5. Generate your API key.
6. Copy the key and save it securely.

![Console](./figures/console.upstage.ai.jpg)

In [81]:
# @title set API key
from pprint import pprint
import os

import warnings

warnings.filterwarnings("ignore")

if "google.colab" in str(get_ipython()):
    # Running in Google Colab. Please set the UPSTAGE_API_KEY in the Colab Secrets
    from google.colab import userdata

    os.environ["UPSTAGE_API_KEY"] = userdata.get("UPSTAGE_API_KEY")
else:
    # Running locally. Please set the UPSTAGE_API_KEY in the .env file
    from dotenv import load_dotenv

    load_dotenv()

assert (
    "UPSTAGE_API_KEY" in os.environ
), "Please set the UPSTAGE_API_KEY environment variable"

In [82]:
from datasets import load_dataset

ds = load_dataset("SkunkworksAI/reasoning-0.01")

In [83]:
ds

DatasetDict({
    train: Dataset({
        features: ['instruction', 'reasoning', 'output', 'reasoning_chains'],
        num_rows: 29857
    })
})

In [84]:
# DatasetDict({
#    train: Dataset({
#        features: ['instruction', 'reasoning', 'output', 'reasoning_chains'],
#        num_rows: 29857
#    })
# })

# Let's take a look at the first five examples in the training set
# Skip Output: {example['output']}
reasoning_chain_examples = ""
for i, example in enumerate(ds["train"].select(range(2))):
    reasoning_chain_examples += f"""----
Example {i + 1}:

Instruction: {example['instruction']}

Reasoning: {example['reasoning']}

Reasoning Chains: {example['reasoning_chains']}
"""

pprint(reasoning_chain_examples)

('----\n'
 'Example 1:\n'
 '\n'
 'Instruction: If a die is rolled three times, what is the probability of '
 'getting a sum of 11? None\n'
 '\n'
 'Reasoning: 1. Understand the problem: We need to find the probability of '
 'getting a sum of 11 when rolling a die three times.\n'
 '2. Calculate total possible outcomes: A die has 6 faces, so for each roll, '
 'there are 6 possibilities. For three rolls, the total possible outcomes are '
 '6^3 = 216.\n'
 '3. Identify favorable outcomes: List all combinations of rolls that result '
 'in a sum of 11. There are 18 such combinations.\n'
 '4. Calculate probability: Divide the number of favorable outcomes by the '
 'total possible outcomes: 18 / 216 = 1/12.\n'
 '5. Conclusion: The probability of getting a sum of 11 when rolling a die '
 'three times is 1/12.\n'
 '\n'
 "Reasoning Chains: [{'step': 1, 'thought': 'Understand the problem: We need "
 'to find the probability of getting a sum of 11 when rolling a die three '
 "times.'}, {'step': 2, 't

In [85]:
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_upstage import ChatUpstage

reasoning_chain_teample = ChatPromptTemplate.from_messages(
    [
        ("human", """Given Instruction, please generate {what}. Please use the following exampels.
    If reasoning and/or reasoning chains are provided, please use them as context to generate the {what}.
    Please only generate the {what} and do not include others.
    
    See the examples below:
    {examples}
    ---
    
    Instruction: {instruction}
    --
    {reasoning}
    --
    {reasoning_chains}   
    """)
    ]
)


promt_example = reasoning_chain_teample.format(
    what="{what}",
    instruction="{instruction}",
    reasoning="{reasoning}",
    reasoning_chains="{reasoning_chains}",
    examples=reasoning_chain_examples,
)
print(promt_example)

Human: Given Instruction, please generate {what}. Please use the following exampels.
    If reasoning and/or reasoning chains are provided, please use them as context to generate the {what}.
    Please only generate the {what} and do not include others.
    
    See the examples below:
    ----
Example 1:

Instruction: If a die is rolled three times, what is the probability of getting a sum of 11? None

Reasoning: 1. Understand the problem: We need to find the probability of getting a sum of 11 when rolling a die three times.
2. Calculate total possible outcomes: A die has 6 faces, so for each roll, there are 6 possibilities. For three rolls, the total possible outcomes are 6^3 = 216.
3. Identify favorable outcomes: List all combinations of rolls that result in a sum of 11. There are 18 such combinations.
4. Calculate probability: Divide the number of favorable outcomes by the total possible outcomes: 18 / 216 = 1/12.
5. Conclusion: The probability of getting a sum of 11 when rolling a

In [86]:
llm = ChatUpstage(model="solar-pro")
chain = reasoning_chain_teample | llm | StrOutputParser()

In [87]:
instruction = "3.11 vs 3.9 which one is greater?"

In [88]:
reasoning = chain.invoke(
    {
        "instruction": instruction,
        "reasoning": "",
        "reasoning_chains": "",
        "what": "reasoning",
        "examples": reasoning_chain_examples,
    }
)

pprint(reasoning)

('Reasoning: 1. Define the criteria for the best place to live: Consider '
 'factors such as safety, cost of living, climate, job opportunities, '
 'education, healthcare, and cultural experiences.\n'
 '2. Research and gather data: Use reliable sources like global indices, '
 'government reports, and reputable websites to gather information on various '
 'locations.\n'
 '3. Create a list of potential destinations: Based on the data, identify a '
 'few places that score well in most or all of the defined criteria.\n'
 '4. Prioritize the criteria: Determine which factors are most important to '
 'you and assign weights to each criterion.\n'
 '5. Evaluate the potential destinations: Apply the weights to the scores of '
 'each location in the defined criteria to calculate an overall score for each '
 'place.\n'
 '6. Consider personal preferences: Take into account your personal '
 'preferences, such as lifestyle, language, and family ties, when evaluating '
 'the potential destinations.\n'

In [89]:
reasoning_chain = chain.invoke(
    {
        "instruction": instruction,
        "reasoning": f"Reasoning: {reasoning}",
        "reasoning_chains": "",
        "what": "reasoning chain",
        "examples": reasoning_chain_examples,
    }
)

pprint(reasoning_chain)

('Reasoning: 1. Define the criteria for the best place to live on Earth: '
 'Consider factors such as safety, cost of living, climate, job opportunities, '
 'education, healthcare, and cultural experiences.\n'
 '2. Research and gather data: Use reliable sources like global indices, '
 'government reports, and reputable websites to gather information on various '
 'locations.\n'
 '3. Create a list of potential destinations: Based on the data, identify a '
 'few places that score well in most or all of the defined criteria.\n'
 '4. Prioritize the criteria: Determine which factors are most important to '
 'you and assign weights to each criterion.\n'
 '5. Evaluate the potential destinations: Apply the weights to the scores of '
 'each location in the defined criteria to calculate an overall score for each '
 'place.\n'
 '6. Consider personal preferences: Take into account your personal '
 'preferences, such as lifestyle, language, and family ties, when evaluating '
 'the potential destina

In [90]:
output = chain.invoke(
    {
        "instruction": instruction,
        "reasoning": f"Reasoning: {reasoning}",
        "reasoning_chains": f"Reasoning Chains: {reasoning_chain}",
        "examples": reasoning_chain_examples,
        "what": "Final answer of the instruction",
    }
)

pprint(output)

('To find the best place to live in the earth, first define the criteria for '
 'the best place to live, which could include factors such as safety, cost of '
 'living, climate, job opportunities, education, healthcare, and cultural '
 'experiences. Then, research and gather data on various locations using '
 'reliable sources. Create a list of potential destinations based on the data, '
 'prioritize the criteria based on your personal preferences, and evaluate the '
 'potential destinations by applying weights to the scores of each location in '
 'the defined criteria. Consider your personal preferences, such as lifestyle, '
 'language, and family ties, when evaluating the potential destinations. If '
 'possible, visit the top-scoring locations to get a firsthand experience of '
 'the places. Finally, make a decision based on the research, evaluation, and '
 'personal experiences, and choose the best place to live on Earth for you.')
